In [19]:
# Importing necessary libraries
import os

import numpy as np

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [20]:
# Defining the device which will be used
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [21]:
# Defining modified LeNet model for testing
class NeutronMonitorDataLeNet(nn.Module):
    def __init__(self):
        super(NeutronMonitorDataLeNet, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Flatten(0)
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=63600, out_features=120),
            nn.ReLU(),
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=3)
        )
        
    def forward(self, x):
        x = self.feature_extractor(x)
        # print(x.shape)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=0)
        return logits, probs

In [22]:
# Loading the model from the snapshot
model = torch.load("/kaggle/input/sopo-neutron-monitor-lenet-model-snapshot/model_snapshot").to(device)
model.eval()

NeutronMonitorDataLeNet(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=0, end_dim=-1)
  )
  (classifier): Sequential(
    (0): Linear(in_features=63600, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=3, bias=True)
  )
)

In [23]:
# Defining a class for the testing dataset
class NeutronMonitorTestingDataset(Dataset):
    def __init__(self):
        super().__init__()
        self._test_files, self._dates = NeutronMonitorTestingDataset._get_neutron_monitor_test_data()
        self._length = len(self._test_files)
    
    @staticmethod
    def _get_neutron_monitor_test_data():
        dir_path = "/kaggle/input/sopo-neutron-monitor-test-data/test_data"
        tests_files_array, dates_array = [], []
        
        _, _, files_with_test_data = next(os.walk(dir_path))
        files_with_test_data.remove(".gitkeep")
        for file in files_with_test_data:
            file_path = os.path.join(dir_path, file)
            dates_array.append(file)
            tests_files_array.append(file_path)
        return tests_files_array, dates_array
    
    def __len__(self):
        return self._length
    
    def __getitem__(self, index):
        sample_date = self._dates[index]
        sample_test_file = self._test_files[index]
        sample_test_data = torch.tensor(
            np.loadtxt(sample_test_file, dtype=np.float64, delimiter=','),
            dtype=torch.float32
        )
        return sample_test_data, sample_date

In [24]:
# Creating dataset and dataloader
testing_dataset = NeutronMonitorTestingDataset()
testing_dataloader = DataLoader(testing_dataset, batch_size=1)

In [25]:
# Evaluating the days using the given model
results = []
for test_data, date in testing_dataloader:
    data_to_evaluate = test_data.to(device)
    _, probs = model(data_to_evaluate)
    _, predicted_label = torch.max(probs, 0)
    if predicted_label == 0:
        prediction = "Calm"
    elif predicted_label == 1:
        prediction = "Weak storm"
    else:
        prediction = "Strong storm"
    results.append({
        "date": date[0],
        "prediction": prediction
    })

In [26]:
# Showing the results
results.sort(key=lambda x: x["date"])
for result_item in results:
    print(f"{result_item['date']} – {result_item['prediction']}")

2022.03.01.txt – Calm
2022.03.02.txt – Calm
2022.03.03.txt – Calm
2022.03.04.txt – Calm
2022.03.05.txt – Strong storm
2022.03.06.txt – Weak storm
2022.03.07.txt – Weak storm
2022.03.08.txt – Weak storm
2022.03.09.txt – Weak storm
2022.03.10.txt – Calm
2022.03.11.txt – Weak storm
2022.03.12.txt – Weak storm
2022.03.13.txt – Strong storm
2022.03.14.txt – Weak storm
2022.03.15.txt – Weak storm
2022.03.16.txt – Calm
2022.03.17.txt – Weak storm
2022.03.18.txt – Calm
2022.03.19.txt – Calm
2022.03.20.txt – Calm
2022.03.21.txt – Weak storm
2022.03.22.txt – Weak storm
2022.03.23.txt – Calm
2022.03.24.txt – Weak storm
2022.03.25.txt – Strong storm
2022.03.26.txt – Calm
2022.03.27.txt – Weak storm
2022.03.28.txt – Calm
2022.03.29.txt – Weak storm
2022.03.30.txt – Strong storm
2022.03.31.txt – Weak storm
